In [1]:
from vllm import LLM, SamplingParams
LOCAL = True
MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"
from functions import *
dtype = 'auto'
gpu_memory_utilization = 0.95

import torch
import pandas as pd
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
llm = LLM(model=MODEL_PATH,
          dtype=dtype,
          enforce_eager=True,
          gpu_memory_utilization=gpu_memory_utilization,
          swap_space=8,
          max_model_len=2048,
          kv_cache_dtype="fp8_e5m2",
          tensor_parallel_size=1)
tokenizer = llm.get_tokenizer()

stop_words = [tokenizer.eos_token if tokenizer is not None and tokenizer.eos_token is not None else '</s>']
stop_words.append("\n")
sampling_params = SamplingParams(temperature=1,
                                 max_tokens=256,
                                 min_tokens=32,
                                 stop=stop_words)

cot_instruction = "\nYou are an expert at mathematical reasoning. Please reason step by step, and put your final answer within \\boxed{}."


n = 3 # beams
n_sol = 3
samples = 7
max_depth = 24
max_pct = 0.8

INFO 05-21 19:45:38 utils.py:253] CUDA_HOME is not found in the environment. Using /usr/local/cuda as CUDA_HOME.
INFO 05-21 19:45:38 config.py:381] Using fp8_e5m2 data type to store kv cache. It reduces the GPU memory footprint and boosts the performance. But it may cause slight accuracy drop. Currently we only support fp8 without scaling factors and make e5m2 as a default format.
INFO 05-21 19:45:38 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='deepseek-ai/deepseek-math-7b-rl', tokenizer='deepseek-ai/deepseek-math-7b-rl', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=True, kv_cache_dtype=fp8_e5m2, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-21 19:45:39 selector.py:16] Using FlashAttention backend.
INFO 05-21 19:45:39 weight_utils.py:177] Using model weights format ['*.safetensors']
INFO 05-21 19:45:45 model_runner.py:104] Loading model weights took 12.8725 GB
INFO 05-21 19:45:46 gpu_executor.py:94] # GPU blocks: 2322, # CPU blocks: 2184


In [3]:
from transformers import LlamaForSequenceClassification
prm_tokenizer = tokenizer
prm_model = LlamaForSequenceClassification.from_pretrained('../Model/PRM_LORA_merge2',\
                                                    num_labels=1,\
                                                    device_map="cpu",
                                                    torch_dtype="auto",
                                                    ).eval()
base_model = prm_model.model
prm_model.score.load_state_dict(torch.load('../Model/model_score2.pth'))

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at ../Model/PRM_LORA_merge2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [4]:
# import json
# with open('../Data/AMC/aime_normal.json', 'r') as file:
#     df = json.load(file)
# # to have consistent format as in Kaggle
# df = pd.DataFrame(df).iloc[:24]
# df.rename(columns={'question': 'problem'}, inplace=True)
import pandas as pd
df = pd.read_csv('../Data/AMC/cleaned_ArtOfProblemSolving.csv')

In [5]:
def process_inputs(inputs):
    # inputs is a list of str
    outs = []
    for problem in inputs:
        query_prompt = problem + cot_instruction
        messages = [{"role": "user","content": query_prompt}]
        input = tokenizer.apply_chat_template(messages, tokenize=False)
        outs.append(input)
    return outs

In [6]:
logit2prob = lambda x: 1/(1+np.exp(-x))
def eval_prm(candidates):
    all_log_probs = []
    for i in range(len(candidates)):
        input_ids = prm_tokenizer.encode(candidates[i], return_tensors="pt").to("cuda")
        with torch.no_grad():
            hidden_states = base_model(input_ids)[0][:,-1] # 1,l,d -> 1,d
            logits = prm_model.score(hidden_states)[0]
        all_log_probs.append(logit2prob(logits.item()))
    return all_log_probs

In [7]:
def is_integer(num):
    if isinstance(num, float):
        return num.is_integer()
    elif isinstance(num, int):
        return True
    else:
        return False
    
def is_between_0_and_999(num):
    return 0 <= num <= 999

import re
def extract_number(text):
    patterns = [
        r'The answer is.*\\boxed\{(.*?)\}',
        r"The answer is[:\s]*\$([0-9]+)\$",
        r"The answer is[:\s]*([0-9]+)"
    ]
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            return match.group(1)
    return 'parse err'

def group_and_sum(A, B):
    '''
    A = ['a','b','a']
    B = [1,2,3]
    -> {'a': 4, 'b': 2}
    '''
    result_dict = {}
    for a, b in zip(A, B):
        if a in result_dict:
            result_dict[a] += b
        else:
            result_dict[a] = b
    return result_dict

def group_and_average(A, B):
    from collections import defaultdict
    # Create a dictionary to store sums and counts for averaging
    sum_dict = defaultdict(lambda: [0, 0])  # Each key maps to [sum, count]
    # Pair elements from A and B and aggregate sums and counts
    for key, value in zip(A, B):
        sum_dict[key][0] += value
        sum_dict[key][1] += 1
    # Calculate averages
    averages = {key: sum_count[0] / sum_count[1] for key, sum_count in sum_dict.items()}
    return averages

def max_dict(d):
    return max(d.items(), key=lambda x: x[1])[0]

def tot_agg(completed_paths):
    answers,scores,_ = zip(*completed_paths)
    if answers:
        groups = group_and_sum(answers, scores)
        return max_dict(groups)
    else:
        return 37 # empty completed_paths
    
def repeat_elements(lst, k):
    return [i for i in lst for _ in range(k)]

def flatten(nested_list):
    """Flatten a nested list."""
    out = []
    lengths = []
    for sublist in nested_list:
        lengths.append(len(sublist))
        for item in sublist:
            out.append(item)
    return out,lengths

def unflatten(flat_list, lengths):
    """Unflatten a flat list into a nested list based on lengths."""
    nested_list = []
    index = 0
    for length in lengths:
        nested_list.append(flat_list[index:index + length])
        index += length
    return nested_list

def filter_input(batch_response,current_level_node):
    # one question filter
    prm_inputs = []
    parents = []
    for candidate,parent in zip(batch_response,current_level_node):
        if candidate.outputs[0].text not in parent:
            prm_input = parent + candidate.outputs[0].text
            prm_inputs.append(prm_input)
            parents.append(parent)
    # Get the indices of unique elements in prm_inputs
    unique_indices = [i for i, x in enumerate(prm_inputs) if prm_inputs.index(x) == i]
    prm_inputs = [prm_inputs[i] for i in unique_indices]
    parents = [parents[i] for i in unique_indices]
    return prm_inputs,parents,len(prm_inputs)

def filter_inputs(batch_responses,current_level_nodes,lengths):
    # all question filter
    # returned value should be flattened
    batch_responses,current_level_nodes = unflatten(batch_responses,lengths),unflatten(current_level_nodes,lengths)
    prm_inputs = []
    lengths = []
    parent_list = []
    uncompleted = [path for path in completed_paths if len(path)<n_sol]
    assert len(batch_responses) == len(uncompleted)
    for batch_response,current_level_node,path in zip(batch_responses,current_level_nodes,uncompleted):
        prm_input,parents,length = filter_input(batch_response,current_level_node)
        if length == 0:# all bad
            while len(path)<n_sol:
                # make complete for this question as there will be no continued effort
                path.append(None)
        else:
            prm_inputs.extend(prm_input)
            parent_list.extend(parents)
            lengths.append(length)
    return prm_inputs,parent_list,lengths

def get_next_node(prm_inputs,prm_scores,completed_paths):
    # need to update completed_paths in-place
    next_level_nodes = []
    combined = list(zip(prm_inputs,prm_scores))
    combined.sort(key=lambda x: x[1], reverse=True)  # Sort nodes by their scores
    max_score = combined[0][1]
    for node,score in combined:
        answer = extract_number(node)
        if answer == 'parse err': # not finished
            if len(next_level_nodes) < n:
                next_level_nodes.append(node)
        else: # finished
            if score > max_score * max_pct:
                try:
                    answer = eval(answer)
                    if is_integer(answer) and is_between_0_and_999(answer):# correct format
                        completed_paths.append((answer,score,node))
                except: # bad eval
                    continue
    return next_level_nodes

def get_next_nodes(prm_inputs,prm_scores,lengths):
    # for completed_paths, next_level_nodes would be removed
    # returned value should be flattened
    prm_inputs,prm_scores = unflatten(prm_inputs,lengths),unflatten(prm_scores,lengths)
    uncompleted = [path for path in completed_paths if len(path)<n_sol]
    assert len(uncompleted) == len(lengths)
    assert len(prm_inputs) == len(lengths)
    assert len(prm_scores) == len(lengths)
    next_level_nodes,lengths = [],[]
    for prm_input,prm_score,completed_path in zip(prm_inputs,prm_scores,uncompleted):
        next_node = get_next_node(prm_input,prm_score,completed_path)
        if len(completed_path) < n_sol:
            next_level_nodes.extend(next_node)
            lengths.append(len(next_node))
    return next_level_nodes,lengths

import gc
def create_llm():
    gc.collect()
    torch.cuda.empty_cache()
    llm = LLM(model=MODEL_PATH,
          dtype=dtype,
          enforce_eager=True,
          gpu_memory_utilization=gpu_memory_utilization,
          swap_space=8,
          max_model_len=2048,
          kv_cache_dtype="fp8_e5m2",
          tensor_parallel_size=1)
    tokenizer = llm.get_tokenizer()
    return llm,tokenizer

In [8]:
# [path for path in completed_paths if len(path)<n_sol] used to track on-going questiones
# flattened inputs, with lengths corresponds to the uncompleted path
# two ways for path to complete, one is via get_next_nodes getting n_sol answers
# two is via filter_inputs, all continuations are bad
import logging
logging.basicConfig(level=logging.ERROR)
current_level_nodes = process_inputs(df.problem.tolist())
lengths = [1] * len(current_level_nodes)
current_level = 1
completed_paths = [[] for _ in current_level_nodes]
data_out = []

while (current_level < max_depth) and (current_level_nodes):
    # everything at this level is flattened
    current_level_nodes = repeat_elements(current_level_nodes,samples)
    lengths = [l*samples for l in lengths]
    batch_responses = llm.generate(current_level_nodes, sampling_params)
    prm_inputs,parent_list,lengths = filter_inputs(batch_responses,current_level_nodes,lengths)
    
    # release VRAM to prm_model
    del llm
    gc.collect()
    torch.cuda.empty_cache()
    prm_model.to('cuda')
    prm_scores = eval_prm(prm_inputs)
    
    # save for Q-learning
    data = list(group_and_average(parent_list,prm_scores).items())
    data_out.extend(data)
    
    # release VRAM to llm
    prm_model.to('cpu')
    llm,tokenizer = create_llm()
    
    current_level_nodes,lengths = get_next_nodes(prm_inputs,prm_scores,lengths)
    current_level += 1

Processed prompts:   7%|▋         | 1558/23723 [01:25<21:30, 17.17it/s]

WARNING 05-21 19:47:23 scheduler.py:245] Input prompt (2742 tokens) is too long and exceeds limit of 2048
WARNING 05-21 19:47:23 scheduler.py:245] Input prompt (2742 tokens) is too long and exceeds limit of 2048
WARNING 05-21 19:47:23 scheduler.py:245] Input prompt (2742 tokens) is too long and exceeds limit of 2048
WARNING 05-21 19:47:23 scheduler.py:245] Input prompt (2742 tokens) is too long and exceeds limit of 2048
WARNING 05-21 19:47:23 scheduler.py:245] Input prompt (2742 tokens) is too long and exceeds limit of 2048
WARNING 05-21 19:47:23 scheduler.py:245] Input prompt (2742 tokens) is too long and exceeds limit of 2048
WARNING 05-21 19:47:23 scheduler.py:245] Input prompt (2742 tokens) is too long and exceeds limit of 2048


Processed prompts:  48%|████▊     | 11488/23723 [08:49<07:26, 27.43it/s]

WARNING 05-21 19:54:47 scheduler.py:245] Input prompt (3981 tokens) is too long and exceeds limit of 2048
WARNING 05-21 19:54:47 scheduler.py:245] Input prompt (3981 tokens) is too long and exceeds limit of 2048
WARNING 05-21 19:54:47 scheduler.py:245] Input prompt (3981 tokens) is too long and exceeds limit of 2048
WARNING 05-21 19:54:47 scheduler.py:245] Input prompt (3981 tokens) is too long and exceeds limit of 2048
WARNING 05-21 19:54:47 scheduler.py:245] Input prompt (3981 tokens) is too long and exceeds limit of 2048
WARNING 05-21 19:54:47 scheduler.py:245] Input prompt (3981 tokens) is too long and exceeds limit of 2048
WARNING 05-21 19:54:47 scheduler.py:245] Input prompt (3981 tokens) is too long and exceeds limit of 2048


Processed prompts: 100%|██████████| 23723/23723 [17:41<00:00, 22.34it/s]


...

WARNING 05-22 08:15:32 scheduler.py:245] Input prompt (2049 tokens) is too long and exceeds limit of 2048
WARNING 05-22 08:15:32 scheduler.py:245] Input prompt (2049 tokens) is too long and exceeds limit of 2048
WARNING 05-22 08:15:32 scheduler.py:245] Input prompt (2049 tokens) is too long and exceeds limit of 2048
WARNING 05-22 08:15:32 scheduler.py:245] Input prompt (2049 tokens) is too long and exceeds limit of 2048
WARNING 05-22 08:15:32 scheduler.py:245] Input prompt (2049 tokens) is too long and exceeds limit of 2048
WARNING 05-22 08:15:32 scheduler.py:245] Input prompt (2049 tokens) is too long and exceeds limit of 2048
WARNING 05-22 08:15:32 scheduler.py:245] Input prompt (2049 tokens) is too long and exceeds limit of 2048


Processed prompts:  70%|██████▉   | 40144/57379 [1:17:31<41:21,  6.94it/s]  

WARNING 05-22 08:54:15 scheduler.py:245] Input prompt (2049 tokens) is too long and exceeds limit of 2048
WARNING 05-22 08:54:15 scheduler.py:245] Input prompt (2049 tokens) is too long and exceeds limit of 2048
WARNING 05-22 08:54:15 scheduler.py:245] Input prompt (2049 tokens) is too long and exceeds limit of 2048
WARNING 05-22 08:54:15 scheduler.py:245] Input prompt (2049 tokens) is too long and exceeds limit of 2048
WARNING 05-22 08:54:15 scheduler.py:245] Input prompt (2049 tokens) is too long and exceeds limit of 2048
WARNING 05-22 08:54:15 scheduler.py:245] Input prompt (2049 tokens) is too long and exceeds limit of 2048
WARNING 05-22 08:54:15 scheduler.py:245] Input prompt (2049 tokens) is too long and exceeds limit of 2048
WARNING 05-22 08:54:15 scheduler.py:245] Input prompt (2049 tokens) is too long and exceeds limit of 2048
WARNING 05-22 08:54:15 scheduler.py:245] Input prompt (2049 tokens) is too long and exceeds limit of 2048
WARNING 05-22 08:54:15 scheduler.py:245] Input

Processed prompts:  72%|███████▏  | 41535/57379 [1:20:46<23:41, 11.15it/s]  

KeyboardInterrupt: 

In [ ]:
out = []
for paths in completed_paths:
    if paths and paths[0]: # not empty or not None
        out.append(tot_agg(paths))
    else:
        out.append(float('-inf'))
print(f"correct {sum([i==j for i,j in zip(df.answer2.tolist(),out)])/df.shape[0]}%")

In [ ]:
import pickle
with open("../llmOutputs/PRM/data_out.pickle", "wb") as f:
    pickle.dump(data_out, f)
with open("../llmOutputs/PRM/completed_paths.pickle", "wb") as f:
    pickle.dump(completed_paths, f)

In [56]:
# with open("../llmOutputs/PRM/completed_paths.pickle", "rb") as f:
#     completed_paths = pickle.load(f)